In [ ]:
import LinearAlgebra
import Random
import BenchmarkTools
import StaticArrays
import NBInclude
import LinearAlgebra
NBInclude.@nbinclude("simulate_ld_motif.ipynb")

# sequence simulator

In [ ]:
# this function takes the length as an argument to generate sequences
# note, this can also be used to generate a motif
function GenerateSequence(Length)
    return collect(Random.randstring("ACGT", Length))
end

In [ ]:
# create a vector of vector{chars}, each with the same length
function GenerateSequences(NumberOfSequences, Length)
    return Sequences = map(i -> GenerateSequence(Length), 1:NumberOfSequences)
end

In [ ]:
function Mutate!(Sequence, Distance)
    # return the sequence if the Distance is 0
    if Distance == 0
        return Sequence
    end
    # choose the sites to mutate
    posToMutate = StatsBase.sample(1:length(Sequence), Distance, replace = false)
    
    # pick the letters that can be used at each position
    basesAtPositions = [string(i) for i in Sequence[posToMutate]]

    lettersToFill = ["ACGT" for i in 1:length(Sequence)]
    
    # figure out what letters each positions can be changed to
    lettersToFill = map((x, y) -> replace(x, y => ""), lettersToFill, basesAtPositions)
    
    # select one character from each of these positions and place them where they should be
    # use only to cast the string to a character
    # https://stackoverflow.com/questions/59946081/julia-convert-string-to-char-or-convert-arraysubstringstring-1-to-char
    Sequence[posToMutate] = map(x -> only(Random.randstring(x, 1)), lettersToFill)
    
    return Sequence
end

In [ ]:
# generate test data
# distance is the number of mutations in each implanted motif
function GenerateTestData_ld(NumberOfSequences, LengthMotif, LengthSequences, Distance)
    # do some error checking to make sure the values provided are valid
    # specifically, Disance <= LengthMotif
    if Distance > LengthMotif
        error("The Distance if larger than the LengthMotif.")
    end

    # LengthMotif <= LengthSequences
    if LengthMotif > LengthSequences
        error("The LengthMotif is longer than the LengthSequences")
    end
    
    # NumberOfSequences >= 2
    if NumberOfSequences <= 1
        error("The NumberOfSequences is 1, which is too small for motif detection")
    end

    # generate the input sequences
    sequences = GenerateSequences(NumberOfSequences, LengthSequences)
    
    # make the motif
    motif = GenerateSequence(LengthMotif)
    
    # generate a mutated motif for each sequence to implant
    motifs = [copy(motif) for i in 1:NumberOfSequences]
    
    # broadcast the mutated motif
    motifs .= Mutate!.(motifs, Distance)
    
    # now implant the mutated motifs and record where we implant it
    motifStarts = rand(1:(LengthSequences - LengthMotif + 1), NumberOfSequences)
    
    # now place the motifs into the sequences vector
    for i in 1:length(sequences)
        sequences[i][motifStarts[i]:(motifStarts[i] + LengthMotif - 1)] = motifs[i]
    end
    
    # now return all the things we may want later
    return (motif = motif, motifs_starts = motifStarts, motifs_implanted = motifs, sequences = sequences)
end

# Score Function

In [ ]:
# it is written in such a way that it is continuously defined
function Score(Starts_inp, Length=length(motif), Seqs=sequences; ln=false)
    # MODIFY the inputs if they are out of bounds :-)
    ### TO DO: return a very large number of the starts are out of bounds
    if (minimum(Starts_inp) < 1) | (maximum(Starts_inp) > (length(Seqs[1]) - Length + 1))
        return(Length * length(Seqs))
    end
    # in the mean time, just fix out of bounds errors

    maxScore = Length * length(Seqs)
        
    seqsMatrix = permutedims(reduce(hcat, map((s, i) -> s[i:(i + Length - 1)], Seqs, Starts_inp)))
    
    # find the most common element in each column
    mostCommon = mapslices(StatsBase.mode, seqsMatrix, dims = 1)
    
    # now count how many sequences are equal to the consensus
    thisScore = sum(map((i, j) -> sum(i .== j), eachslice(seqsMatrix, dims = 2), mostCommon))
    
    if ln
        return(-1 * log(maxScore - thisScore + 1))
    else
    # make the minimum (best) score 1 so we can compute the log
        return(maxScore - thisScore + 1)
    end
end

In [ ]:
# calculate a partial score in the plus and minus direction
# return positive if going in the positive direction increases the score
# return negative if going in the negative direction increases the score
# the value given is the sum score difference gained from going x-1 -> x -> x+1
#### TO DO: deal with edges of the search space
# https://www2.atmos.umd.edu/~ekalnay/syllabi/AOSC614/NWP-CH03-2-2.pdf
function Grad(Starts_inp, seq, dt, k)
    backward = copy(Starts_inp)
    forward = copy(Starts_inp)
    
    backward[seq] -= dt
    forward[seq] += dt
    
    #score1 = Score(Starts_inp) - Score(backward)
    #score2 = Score(forward) - Score(Starts_inp)
    return((Score(forward, k; ln = true) - Score(backward, k; ln = true))/(2*dt))
end

In [ ]:
function CalcGrad(StartsInp, dt, k)
    return Grad.((StartsInp,), 1:length(StartsInp), (dt,), (k,))
end

In [ ]:
function Grad2(Starts_inp, seq, dt=1)
    backward = copy(Starts_inp)
    forward = copy(Starts_inp)
    
    backward[seq] -= dt
    forward[seq] += dt
    
    #score1 = Score(Starts_inp) - Score(backward)
    #score2 = Score(forward) - Score(Starts_inp)
    return(Score(forward; ln = true) - 2 * Score(Starts_inp; ln = true) + Score(backward; ln = true)/(dt^2))
end

In [ ]:
function CalcGrad2(StartsInp, dt=1)
    return Grad2.((StartsInp,), 1:length(StartsInp), (dt,))
end

# HMC

In [ ]:
function H(particle, velocity)
    return(U(particle) + 1/2 * velocity' * inv(LinearAlgebra.I) * velocity)
end

In [ ]:
function U(particle)
    return(-1 * log(Score(particle)))
end

In [ ]:
function leap(particle, velocity, dt, iter, k, Seqs)
    # TODO: preallocate the array of static arrays
    particle_sample = []
    
    #### To do: convert these vectors to staticarrays
    # initial particle position and velocity
    xnt = copy([i for i in particle])
    pnt = copy([i for i in velocity])
    
    # iterate for the number of time steps desired
    # TODO: add stopping criterion
    for t in 1:iter
        # do the first half-step
        # update the particles momentum by looking at the previous moments
        # and adding half-ish the change in the potential energy
        pn_t_dt2 = pnt .- (dt/2) .* CalcGrad(xnt, dt, k)
        
        # update the particle position after t time
        # by using the new momentum, found in the previous half step
        xn_t_dt = xnt .+ dt * inv(LinearAlgebra.I) * pn_t_dt2
        # round this value to an integer, since our scorer only works
        # on discrete values
        
        ### TODO: add a bounds check here to prevent from going out of bounds
        # also, do something different besides rounding, like using the logic in add_vectors
        xn_t_dt = Int.(round.(xn_t_dt))
        
        # update the momentum again, this time by evaluating the gradient at the new point
        pn_t_dt = pn_t_dt2 .- (dt/2) .* CalcGrad(xn_t_dt, dt, k)
        
        # do a metropolis-hastings step
        
        # compute the ratio of hamiltonian new/old like on the wiki page      
        # if this value is off substantially, we will end up rejecting the sample
        α = min(1, (exp(-1 * H(xn_t_dt, pn_t_dt)))/(exp(-1 * H(xnt, pnt))))

        # if our random number is less than alpha, accept the new proposal
        if (rand(1)[1] <= α) & (minimum(xn_t_dt) >= 1) | (maximum(xn_t_dt) >= (length(Seqs[1]) - k + 1))
            xnt = xn_t_dt
            pnt = pn_t_dt
            append!(particle_sample, (xnt,))
        end
        
        #### TODO:
        # check if we are at a boundary. if so, reverse the sign of the velocity for that dimension
    end
    return(particle_sample)
end

# PSO

In [ ]:
function add_vectors(vec1, vec2)
    # find the norm of the first vector, which is the particle
    added = (vec1 .+ vec2).*(LinearAlgebra.norm(vec1, 2)/(LinearAlgebra.norm(vec1, 2)+LinearAlgebra.norm(vec2, 2)))
    
    # get the remainder part
    round_dir = ifelse.(added - floor.(added) .< rand(length(vec1)), RoundUp::RoundingMode, RoundDown::RoundingMode)
    # add the two vectors together
    # now round each value to get an integer array
    # get a random number for each position in the vector

    vec_out = StaticArrays.SVector{length(vec1), Int64}(map((x, y) -> round(Int64, x, y), added, round_dir))
    return vec_out
    ## todo: deal with situations where the norm of vec1 is very close tozero
end

In [ ]:
function Algorithm(seqs, k, dims = 3, nparticles = 10, maxiter = 500)
    # generate the starting points for the algorithm
    # store these in a matrix, where each column is a particle, and each row is a dimension in the search space
    particles = [StaticArrays.SVector{dims, Int64}(rand(1:(length(seqs[1]) - k + 1), dims)) for i in 1:nparticles]
    velocities = [StaticArrays.SVector{dims, Int64}(rand(-3:3, dims)) for i in 1:nparticles]
    direction = [StaticArrays.SVector{dims, Int64}(zeros(Int64, dims)) for i in 1:nparticles]
    scores = zeros(Int64, nparticles)

    #scores = StaticArrays.@MVector zeros(Int64, nparticles)
    
    # create a set to store the points that we have visited, to avoid looking at the same points repeatedly
    visited = Set{StaticArrays.SVector{dims, Int64}}()
    
    # give a size hint for the set of visited points, hopefully preventing it from growing repeatedly
    sizehint!(visited, nparticles * maxiter)
    
    iter = 1
    
    best_particle = zeros(Int64, dims)
    global_optimum = Inf
        
    wiggled = 0
    terminated = 0
    # loop until convergence
    # while iter < maxiter
    while iter <= maxiter
        # "wiggle" each particle position if it's already been visited, or if its out of bounds
        idx = findall(x -> (x in visited) | (minimum(x) < 1) | (maximum(x) > (length(seqs[1]) - k + 1)), particles)
        
        # for each of these, try wiggling
        for part in idx
            wiggled += 1
            newpart = particles[part]
            tries = 1
            while (tries < 40) & (newpart in visited)
                dist = Int(floor(tries/4 + 1))
                newpart = particles[part] .+ (rand(1:dist, dims) .* rand([-1, 1], dims))
                tries += 1
            end
            particles[part] = newpart
        end
        
        # compute the current scores of each particle
        map!(x -> Score(x), scores, particles)
            
        # find the current best particle
        best_score = findmin(scores)
        
        # if this score is better than the current best score, update
        if best_score[1] < global_optimum[1]
            global_optimum = best_score[1]
            best_particle .= particles[best_score[2]]
        end
        
        ##### TODO: change this line that it actually uses the information from HMC
        # we will get a sample of particle positions from HMC
        # we can "thin" our sample to prefer the proposed positions that are in the direction of the best particle
        #####
        
        # for each of the current vectors
            # sample nearby positions and run HMC for 30 steps for each of 30 sampled particles
            # sample such that particles are more likely to start in the direction the particle is heading and in the direction of the best particle
        
        # perform local search with HMC:
        #leap([10, 10, 10] , [1, 12, 1], 1, 30, 10, seqs)
        
        # add the vectors and update the velocities
        # adding the vector of the current particle and the direction that we want to head
        map!((x, y) -> add_vectors(x, sign.(best_particle .- y)), velocities, velocities, particles)
        
        # save the new locations we've visited
        map(x -> push!(visited, x), velocities, )

        # update the particle positions based on the velocities
        particles .+= velocities
        
        # convergence criteria: quit when idx is full for ten iterations in a row
        if length(idx) == nparticles
            terminated += 1
        end
        
        if terminated == 10
            iter += maxiter
        end

        iter += 1
    end 
    # print(wiggled)
    return(global_optimum, best_particle)
end

# Unit Testing

In [ ]:
# test add_vectors!

In [ ]:
# check if out of bounds is working in Score
# motif, motif_starts, motifs_implanted, sequences = GenerateTestData_ld(3, 14, 30, 0)
# Score([1, 1, 0], sequences, 14)
# Score([1, 1, 18], sequences, 14)

# Hyperparameter Tuning

# Benchmarking

In [ ]:
# set the seed
Random.seed!(100)
# use tuple unpacking to get some test values
# NumberOfSequences, LengthMotif, LengthSequences, Distance
motif, motif_starts, motifs_implanted, sequences = GenerateTestData_ld(3, 10, 30, 0)

In [ ]:
#Random.seed!(123)
Algorithm(sequences, 14, size(sequences, 1), 100, 500)